---
title: Testing the Enformer pipeline with added parameters for personalized prediction on rats
author: Sabrina Mi
date: 8/12/23
---

## Test for a single individual and gene

We chose `ENSRNOG00000054549`, centered at the TSS chr20:12118762.


```
conda activate enformer-predict-tools

cd /Users/sabrinami/Github/shared_pipelines/enformer_pipeline

python scripts/enformer_predict.py --parameters /Users/sabrinami/Github/deep-learning-in-genomics/posts/2023-08-12-test-run-of-personalized-enformer-pipeline-for-rats/local_test_personalized.json

```


